<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Pedro%20Gengo/Pedro_Gengo_Aula_9_Exerc%C3%ADcio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Pedro Gabriel Gengo Lourenço'
print(f'Meu nome é {nome}')

Meu nome é Pedro Gabriel Gengo Lourenço


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, dada as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.2 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Thu Jun  2 11:16:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List
from tqdm.notebook import tqdm


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.max_seq_length = max_seq_length
        self.tokenized_texts = []
        for text in tqdm(texts):
          tokenized_text = tokenize(f'[CLS] {text}', tokenizer) # + [tokenizer.vocab['[SEP]']]  # Inserindo token CLS e SEP
          tokenized_text += [tokenizer.vocab['[PAD]']] * max(0, 1 + max_seq_length - len(tokenized_text))
          for i in range(0, len(tokenized_text) - 1, max_seq_length):
            if i + max_seq_length < len(tokenized_text):
              self.tokenized_texts.append(tokenized_text[i: i + max_seq_length + 1])
            else:
              self.tokenized_texts.append(tokenized_text[-max_seq_length - 1:])
        self.tokenized_texts = torch.LongTensor(self.tokenized_texts)

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        x_y = self.tokenized_texts[idx]
        return x_y[:-1], x_y[1:]

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


In [8]:
tokenize('Eu gosto de correr e de comer muita pizza', tokenizer)

[3396, 10303, 125, 13239, 122, 125, 1847, 5747, 13779, 15616]

In [9]:
dummy_texts = ['Eu gosto de correr e de comer muita pizza', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 3
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,  125, 13239,  122,    125,  1847,  5747],
     [  3396, 10303, 125, 13239,   122,  125,   1847,  5747, 13779],
     [  101,  1660,  5971,  785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,  125, 13239,  122,    125,  1847,  5747, 13779],
     [ 10303, 125, 13239,   122,  125,   1847,  5747, 13779, 15616],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [10]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-02 11:16:38--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G  65.5MB/s    in 18s     

2022-06-02 11:16:56 (66.8 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [83]:
# Load datasets
max_seq_length = 9

train_examples = 50000
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 50200 lines.


  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [84]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 6186062
valid examples: 13933
test examples: 13960


In [133]:
from collections import OrderedDict

class MultiHeadSelfAttention(torch.nn.Module):

  def __init__(self, max_seq_length, embedding_dim, num_heads, padding_id):
    super().__init__()
    self.num_heads = num_heads
    self.padding_id = padding_id
    self.max_seq_length = max_seq_length
    self.W_q = torch.nn.Linear(embedding_dim, embedding_dim, bias = False)
    self.W_k = torch.nn.Linear(embedding_dim, embedding_dim, bias = False)
    self.W_v = torch.nn.Linear(embedding_dim, embedding_dim, bias = False)
    self.W_o = torch.nn.Linear(embedding_dim, embedding_dim, bias = False)
    self.mask = torch.tril(torch.ones(max_seq_length, max_seq_length)).unsqueeze(0).to(device)
  
  def _self_attention(self, q, k, v, mask):
    s = torch.matmul(q, k.transpose(-2, -1))  # B, H, L, D/H x B, H, D/H, L -> B, H, L, L   Tenho a ponderação de todos os tokens contra todos
    s = s.masked_fill(mask.unsqueeze(1) == 0, -float("inf"))
    p = torch.nn.functional.softmax(s, dim=-1)  # shape = B, H, L, L
    att_output = torch.matmul(p, v)  # B, H, L, L x B, H, L, D/H -> B, H, L, D/H
    return att_output.transpose(1, 2) # B, L, H, D/H  Para cada token, vou ter uma nva representação dele
  
  def forward(self, inputs, mask):
    batch_size = inputs.shape[0]
    q = self.W_q(inputs)  # shape = B, L, D
    q = q.view(batch_size, self.max_seq_length, self.num_heads, -1)  # shape = B, L, H, D/H
    q = q.transpose(1, 2)  # shape = B, H, L, D/H
    k = self.W_k(inputs)  # shape = B, L, D
    k = k.view(batch_size, self.max_seq_length, self.num_heads, -1)
    k = k.transpose(1, 2)  # shape = B, H, L, D/H
    v = self.W_v(inputs)  # shape = B, L, D
    v = v.view(batch_size, self.max_seq_length, self.num_heads, -1)
    v = v.transpose(1, 2)  # shape = B, H, L, D/H
    
    att_output = self._self_attention(q, k, v, mask)
    att_output = att_output.reshape(batch_size, self.max_seq_length, -1)  # B, L, D  Cada token tem sua representação contextualizada
    att_output = self.W_o(att_output)
    return att_output

In [134]:
batch_size_ = 2
max_seq_length_ = 3
padding_id_ = 101
inputs_ = torch.tensor([[1, 2, 101], [3, 4, 5]])

In [135]:
mask = torch.tril(torch.ones(batch_size_, max_seq_length_, max_seq_length_))
mask = mask.masked_fill(inputs_.unsqueeze(1) == padding_id_, 0)
mask

tensor([[[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 0.]],

        [[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]]])

In [136]:
torch.nn.functional.softmax(mask[0], dim=-2)

tensor([[0.3333, 0.1554, 0.3333],
        [0.3333, 0.4223, 0.3333],
        [0.3333, 0.4223, 0.3333]])

In [137]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()
        self.pad_token_id = pad_token_id
        self.max_seq_length = max_seq_length
        self.normal_embds = torch.nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
        self.positional_embds = torch.nn.Embedding(max_seq_length, dim, padding_idx=pad_token_id)
        self.head1 = MultiHeadSelfAttention(max_seq_length, dim, n_layers, pad_token_id)
        self.head2 = MultiHeadSelfAttention(max_seq_length, dim, n_layers, pad_token_id)
        self.classifier = torch.nn.Sequential(
            OrderedDict([
                         ('layer_1', torch.nn.Linear(dim, dim*2)),
                         ('relu', torch.nn.ReLU()),
                         ('layer_2', torch.nn.Linear(dim*2, vocab_size, bias=False))
            ])
        )

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0]
        normal_embds = self.normal_embds(inputs)
        embds = normal_embds + self.positional_embds.weight  # shape = B, L, D
        
        mask = torch.tril(torch.ones(batch_size, self.max_seq_length, self.max_seq_length)).to(device)
        mask = mask.masked_fill(inputs.unsqueeze(1) == self.pad_token_id, 0)

        att_out = self.head1.forward(embds, mask)  # shape = B, L, D
        att_out_ = att_out + normal_embds  # Residual connection
        att_out = self.head2.forward(att_out_, mask)

        att_out = att_out + att_out_  # Residual connection

        output = self.classifier(att_out)  # shape = B, L, D
        return output

## Teste o modelo com um exemplo

In [138]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset, batch_size=1)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [139]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5762112


## Assert da Perplexidade


In [140]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
        target: a LongTensor of shape (batch_size, seq_len)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              31199
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [159]:
max_examples = 300_000_000
eval_every_steps = 1000
lr = 3e-4


# model = LanguageModel(
#     vocab_size=tokenizer.vocab_size,
#     max_seq_length=max_seq_length,
#     dim=256,
#     n_layers=4,
#     pad_token_id=tokenizer.pad_token_id,
# ).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()
    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 138.49, valid ppl: 155.00
1000 steps; 1024000 examples so far; train ppl: 148.82, valid ppl: 161.79
2000 steps; 2048000 examples so far; train ppl: 148.24, valid ppl: 159.71
3000 steps; 3072000 examples so far; train ppl: 147.34, valid ppl: 158.25
4000 steps; 4096000 examples so far; train ppl: 146.56, valid ppl: 155.78
5000 steps; 5120000 examples so far; train ppl: 145.49, valid ppl: 154.57
6000 steps; 6144000 examples so far; train ppl: 144.81, valid ppl: 153.16
7000 steps; 7168000 examples so far; train ppl: 138.77, valid ppl: 153.31
8000 steps; 8192000 examples so far; train ppl: 138.99, valid ppl: 152.39
9000 steps; 9216000 examples so far; train ppl: 138.70, valid ppl: 151.11
10000 steps; 10240000 examples so far; train ppl: 138.23, valid ppl: 149.80
11000 steps; 11264000 examples so far; train ppl: 137.97, valid ppl: 149.19
12000 steps; 12288000 examples so far; train ppl: 137.77, valid ppl: 147.46
13000 steps; 13312000 examples so far; tr

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [160]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 134.22589643273025


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [161]:
prompt = '[CLS] Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)  # O(n) -> Assumindo uma lookup table
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.  O(1)
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia  O(1)
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()  # O(n)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)  # O(n + 1)
    print(prompt)

[CLS] Eu gosto de comer pizza pois me faz sentir
[CLS] Eu gosto de comer pizza pois me faz sentir,
[CLS] Eu gosto de comer pizza pois me faz sentir, mas
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus pais
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus pais.
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus pais..
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus pais...
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus pais....
[CLS] Eu gosto de comer pizza pois me faz sentir, mas não é um dos meus pais.....
[CLS] Eu gosto de comer pizza pois me faz sentir, 

In [176]:
prompt = 'Queria viajar para a Europa e ir conhecer o'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)  # O(n) -> Assumindo uma lookup table
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.  O(1)
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia  O(1)
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()  # O(n)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)  # O(n + 1)
    print(prompt)

Queria viajar para a Europa e ir conhecer o país
Queria viajar para a Europa e ir conhecer o país.
Queria viajar para a Europa e ir conhecer o país. O
Queria viajar para a Europa e ir conhecer o país. O que
Queria viajar para a Europa e ir conhecer o país. O que é
Queria viajar para a Europa e ir conhecer o país. O que é o
Queria viajar para a Europa e ir conhecer o país. O que é o caso
Queria viajar para a Europa e ir conhecer o país. O que é o caso de
Queria viajar para a Europa e ir conhecer o país. O que é o caso de um
Queria viajar para a Europa e ir conhecer o país. O que é o caso de um país
Queria viajar para a Europa e ir conhecer o país. O que é o caso de um país que
Queria viajar para a Europa e ir conhecer o país. O que é o caso de um país que não
Queria viajar para a Europa e ir conhecer o país. O que é o caso de um país que não é
Queria viajar para a Europa e ir conhecer o país. O que é o caso de um país que não é um
Queria viajar para a Europa e ir conhecer o país. O que 

In [163]:
prompt = 'Querido Papai Noel, esse ano eu gostaria de ganhar um'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)  # O(n) -> Assumindo uma lookup table
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.  O(1)
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia  O(1)
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()  # O(n)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)  # O(n + 1)
    print(prompt)

Querido Papai Noel, esse ano eu gostaria de ganhar um pouco
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano.
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano. A
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano. A primeira
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano. A primeira vez
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano. A primeira vez que
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano. A primeira vez que o
Querido Papai Noel, esse ano eu gostaria de ganhar um pouco mais de um ano. A primeira vez que o governo
Querido Papai Noel, esse ano eu 

In [164]:
texts[-1]

'ISO 9001 ISO 9001 Quality Management System Em 2009 a Rodoeng Transportes Rodoviários Ltda, certificou-se na norma ISO 9001:2008 – Sistema de Gestão da Qualidade, ratificando nossa experiência de mais de três décadas em transportes de carga para todo o território nacional e garantindo credibilidade aos serviços prestados. As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços, monitoram o ambiente de trabalho, o aumento da satisfação dos colaboradores e dos clientes. Assim, fortalecem o clima organizacional ao mesmo tempo em que ganham em produtividade, aumentando sua vantagem competitiva. A Qualidade, Eficiência e a Satisfação de nossos clientes são nossas prioridades na busca pelo crescimento. … Política da Qualidade "Suprir as necessidades dos nossos clientes através de treinamento dos colaboradores, da parceria com fornecedores e da melhoria contínua dos serviços, tendo como foco o compromisso com a saúde e segurança dos colaboradores, a preserva

In [165]:
prompt = 'As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)  # O(n) -> Assumindo uma lookup table
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.  O(1)
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia  O(1)
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()  # O(n)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)  # O(n + 1)
    print(prompt)

As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços.
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A empresa
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A empresa de
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A empresa de telefonia
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A empresa de telefonia fixa
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A empresa de telefonia fixa,
As empresas certificadas na ISO 9001 aperfeiçoam o desempenho de seus produtos e serviços. A empresa de telefonia fixa, o
As empresas certificadas na ISO 9001 aperfeiço

In [177]:
prompt = 'Gosto muito de Orlando, por isso quero ir conhecer os parques da'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)  # O(n) -> Assumindo uma lookup table
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.  O(1)
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia  O(1)
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()  # O(n)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)  # O(n + 1)
    print(prompt)

Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade.
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é o
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é o Brasil
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é o Brasil,
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é o Brasil, a
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é o Brasil, a partir
Gosto muito de Orlando, por isso quero ir conhecer os parques da cidade. O que é o Brasil, a partir de
Gosto muito de Orlando, por isso quero ir conhecer os parque

In [182]:
prompt = 'pato ceu sol chao huashua salsla rato'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)  # O(n) -> Assumindo uma lookup table
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.  O(1)
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia  O(1)
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()  # O(n)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)  # O(n + 1)
    print(prompt)

pato ceu sol chao huashua salsla rato,
pato ceu sol chao huashua salsla rato, e
pato ceu sol chao huashua salsla rato, e o
pato ceu sol chao huashua salsla rato, e o que
pato ceu sol chao huashua salsla rato, e o que é
pato ceu sol chao huashua salsla rato, e o que é o
pato ceu sol chao huashua salsla rato, e o que é o que
pato ceu sol chao huashua salsla rato, e o que é o que é
pato ceu sol chao huashua salsla rato, e o que é o que é o
pato ceu sol chao huashua salsla rato, e o que é o que é o que
pato ceu sol chao huashua salsla rato, e o que é o que é o que é
pato ceu sol chao huashua salsla rato, e o que é o que é o que é o
pato ceu sol chao huashua salsla rato, e o que é o que é o que é o que
pato ceu sol chao huashua salsla rato, e o que é o que é o que é o que é
pato ceu sol chao huashua salsla rato, e o que é o que é o que é o que é o
pato ceu sol chao huashua salsla rato, e o que é o que é o que é o que é o que
pato ceu sol chao huashua salsla rato, e o que é o que é o que é o

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.

$O(max\_output\_tokens * (prompt + ))$